### Arches QuerySets tutorial

This tutorial shows the approach `arches-querysets` uses to adapt the Arches data model to the Django ORM and the rest of the Django ecosystem.

#### Test data
We'll use a sample resource model called "Datatype Lookups". Set up the test data. You should replace the settings module below with the one specifiying the database you wish to use for this tutorial. Run `manage.py setup_db` first if necessary. (This is not included in the notebook out of caution.)

>[!TIP]
> If you're having trouble, be sure your IDE has the python interpreter for your virtual environment selected.

In [1]:
import os

import django
from django.core.management import call_command

# Simulate manage.py
os.environ["DJANGO_SETTINGS_MODULE"] = "arches_querysets.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

call_command("delete_test_data")
call_command("add_test_data")

Deleting test data...
Finished!
Creating test data...
Finished!


#### Design

The system allows you to request resources and tiles with node values aliased by their node aliases, and with tiles nested under the alias for that child nodegroup's grouping node. This reflection of the `TileModel.data` attribute becomes a new attribute `aliased_data`.

This is done by subclassing the `ResourceInstance` and `TileModel` models from core Arches. Because they provide methods to retrieve and query nested tiles, the names of the proxy models are:
- `ResourceInstance` -> `ResourceTileTree`
- `TileModel` -> `TileTree`

> [!NOTE]
> These models do not subclass the proxy models `Resource` and `Tile` from core Arches, because, like those models, these models have their own distinct approach for fetching data.

In TypeScript notation, the nesting of data looks as follows:

```ts
interface ResourceTileTree {
    aliased_data: AliasedData;
    resourceinstanceid: string | null;
    name: string;
    // further resourceinstance fields...
}

interface TileTree {
    aliased_data: AliasedData;
    nodegroup: string;
    parenttile: string | null;
    provisionaledits: object | null;
    resourceinstance: string;
    sortorder: number;
    tileid: string | null;
}

interface AliasedData {
    [node_alias: string]: AliasedNodeData | AliasedNodegroupData;
}

interface AliasedNodeData {
    display_value: string;
    node_value: unknown;
    details: unknown[];
}

type AliasedNodegroupData = TileTree | TileTree[] | null;
```

Of note:
- The key-value pairs on any `AliasedData` representation will be a mixture of single nodes in the current nodegroup and nodegroup aliases for child nodegroups that unpack to nested tiles.
- Cardinality-1 nodegroups will be represented as `null` or a `TileTree`.
- Cardinality-n nodegroups will be represented as a possibly empty array of `TileTree`s.
- Node values are always grouped under the alias of the grouping node for the nodegroup ("nodegroup alias"). That is, no data-collecting nodes appear will appear *directly* under the tile.
- Node values are represented as an object of three key-value pairs. `display_value` and `details` are read-only.

#### Entrypoint: get_tiles()

The `ResourceTileTree` and `TileTree` models have [custom model managers](https://docs.djangoproject.com/en/5.2/topics/db/managers/#custom-managers) that encapsulate the custom querying behavior.

They both provide a `get_tiles()` entrypoint. This method allows you to use node aliases as ORM lookups and tells the ORM how to shape the data once it is fetched.

Query for the two test resources. One test resource has node values referencing the number 42 for most nodes; the other has a value of `None` in every node:

In [3]:
from arches_querysets.models import *

ResourceTileTree.get_tiles(graph_slug="datatype_lookups")

<ResourceTileTreeQuerySet [<ResourceTileTree: Datatype Lookups: Resource referencing 42 (9cf083a4-3d7b-43c2-b755-f40885cb3b9c)>, <ResourceTileTree: Datatype Lookups: Resource referencing None (3e5684df-cca1-4db1-b859-90a2902f38bb)>]>

Filter on node alias values to exclude the resource with only None for node values. The node aliases for the test model follow this pattern:
- number_alias_1
- number_alias_n
- number_alias_1_child
- number_alias_n_child

In [4]:
from pprint import pprint

resource_with_data = ResourceTileTree.get_tiles(graph_slug="datatype_lookups").exclude(number_alias=None).get()
pprint(resource_with_data.aliased_data)
# This resource has two top nodegroups: "datatypes_1" and "datatypes_n"

AliasedData(datatypes_1=<TileTree: datatypes_1 (dcb8bce8-cdd4-4498-aa6d-3454ff66c0c1)>,
            datatypes_n=[<TileTree: datatypes_n (7b7d4c9f-e809-47d6-91be-bcce161f2f5a)>])


Inspect each tile's `aliased_data`:

In [5]:
pprint(resource_with_data.aliased_data.datatypes_1.aliased_data)
pprint(resource_with_data.aliased_data.datatypes_n[0].aliased_data)

AliasedData(concept_alias=<Value: Value object (d8c60bf4-e786-11e6-905a-b756ec83dad5)>,
            geojson_feature_collection_alias=None,
            concept_list_alias=[<Value: Value object (d8c60bf4-e786-11e6-905a-b756ec83dad5)>],
            number_alias=42,
            resource_instance_alias=<ResourceInstance: Datatype Lookups: Resource referencing 42 (9cf083a4-3d7b-43c2-b755-f40885cb3b9c)>,
            resource_instance_list_alias=[<ResourceInstance: Datatype Lookups: Resource referencing 42 (9cf083a4-3d7b-43c2-b755-f40885cb3b9c)>],
            file_list_alias=[{'accepted': True,
                              'altText': {'en': {'direction': 'ltr',
                                                 'value': 'Illustration of '
                                                          'recent '
                                                          'accessibility '
                                                          'improvements'}},
                              'attributio

Notice some node values are richer than the pure tile representation. For instance, this concept node value is a `Concept` instance rather than a `UUID`:

In [6]:
print(resource_with_data.aliased_data.datatypes_1.aliased_data.concept_alias)

Value object (d8c60bf4-e786-11e6-905a-b756ec83dad5)


This happens because the `ConceptDataType` registered by arches-querysets in its `datatypes` directory has a `to_python()` method that is called when materializing the QuerySet. As a project implementer, you can register your own datatypes to provide similar functionality, e.g. to provide a `LocalizedString` class for string values.

If you don't need these richer Python representations--perhaps because you simply care about the display value--pass `as_representation=False`, as the Django REST Framework (DRF) integration always does, to get the "three-key" object mentioned above:

> [!NOTE]
> Depending on the datatype, getting the representation instead of the python object may be more or less performant.

In [7]:
resource_as_repr = ResourceTileTree.get_tiles(
    graph_slug="datatype_lookups", as_representation=True
).exclude(number_alias=None).get()

print(resource_as_repr.aliased_data.datatypes_1.aliased_data.concept_alias)

{'node_value': 'd8c60bf4-e786-11e6-905a-b756ec83dad5', 'display_value': 'Arches', 'details': [{'concept_id': '00000000-0000-0000-0000-000000000001', 'language_id': 'en', 'value': 'Arches', 'valueid': 'd8c60bf4-e786-11e6-905a-b756ec83dad5', 'valuetype_id': 'prefLabel'}]}


#### Filtering

`.get_tiles()` fetches node aliases for the requested graph, prepares ORM expressions for each one, and passes the expressions to [`QuerySet.alias()`](https://docs.djangoproject.com/en/5.2/ref/models/querysets/#alias) so that the node alias can be used like any standard Django field name.

>[!NOTE]
> `.alias()` is like `.annotate()`, but lazier: if you don't actually *use* the alias in a filter, order by, or aggregate, Django just drops the alias entirely instead of including it in the SQL.

>[!NOTE]
> Fetching node aliases for a graph is the only database query done inside `.get_tiles()`. If your code calls `.get_tiles()` multiple times, for better performance you can provide an iterable of node instances yourself via the `nodes` argument.

Filter on the `string_alias` node, using [key, index, and path transforms](https://docs.djangoproject.com/en/5.2/topics/db/queries/#key-index-and-path-transforms) to interrogate JSON values:

In [8]:
test_resources = ResourceTileTree.get_tiles("datatype_lookups")
resource = test_resources.filter(string_alias__en__value__startswith="forty").first()
resource.aliased_data.datatypes_1.aliased_data.string_alias

{'en': {'value': 'forty-two', 'direction': 'ltr'}}

But what if you want a `startswith` search on a value in *any* language? arches-querysets registers additional datatype-specific lookups, such as `any_lang_startswith` for localized strings, that can be used with the Django `__` syntax for joining lookups:

In [9]:
resource = test_resources.filter(string_alias__any_lang_startswith="forty").first()
resource.aliased_data.datatypes_1.aliased_data.string_alias

{'en': {'value': 'forty-two', 'direction': 'ltr'}}

The following custom lookups are available as of 96b8e802d367b53c786e2bc02fdd121835b5e008:


String datatype (cardinality 1 & cardinality N):
`any_lang_contains`
`any_lang_icontains`
`any_lang_startswith`
`any_lang_istartswith`

Cardinality-N non-localized-string:
`any_contains`
`any_icontains`

Resource Instance datatype:
`id`

Cardinality-N resource instance:
`ids_contain`

Resource Instance list datatype:
`contains`

Cardinality-N resource instance list:
`ids_contain`

>[!NOTE]
> Custom lookups are defined in `lookups.py` and tested in `test_lookups.py`.

For resources, *shallow* queries on nodes for nested tiles are possible without needing to specify the parents:

In [10]:
resource = test_resources.filter(non_localized_string_alias_child__isnull=False).first()
child_tile = resource.aliased_data.datatypes_1.aliased_data.datatypes_1_child
print(child_tile.aliased_data.non_localized_string_alias_child)

child-1-value


From there, you can backtrack to the parent:

In [11]:
child_tile.parent

<TileTree: datatypes_1 (dcb8bce8-cdd4-4498-aa6d-3454ff66c0c1)>

#### Updating
You can save back any value accepted by the datatype's `transform_value_for_tile()`:

This example uses `.save(force_admin=True)` because without, `save()` falls back to the anonymous user, which lacks Resource Editor permissions. (The edits might seem to "go missing", but they actually end up in provisional edits.) In a request/response cycle, provide `user=request.user`. (The DRF API handles all this for you.)

In [12]:
resource.aliased_data.datatypes_1.aliased_data.string_alias = 'forty-three'
resource.save(force_admin=True)
resource.aliased_data.datatypes_1.aliased_data.string_alias

{'en': {'value': 'forty-three', 'direction': 'ltr'}}

In [13]:
resource.aliased_data.datatypes_1.aliased_data.string_alias = {'en': {'value': 'forty-four', 'direction': 'ltr'}}
resource.save(force_admin=True)
resource.aliased_data.datatypes_1.aliased_data.string_alias

{'en': {'value': 'forty-four', 'direction': 'ltr'}}

In [14]:
resource.aliased_data.datatypes_1.aliased_data.string_alias = {
    "display_value": "",  # ignored
    "node_value": {'en': {'value': 'forty-five', 'direction': 'ltr'}},
    "details": [],
}
resource.save(force_admin=True)
resource.aliased_data.datatypes_1.aliased_data.string_alias

{'en': {'value': 'forty-five', 'direction': 'ltr'}}

Those three calls were equivalent. This one fails, though:

In [15]:
resource.aliased_data.datatypes_1.aliased_data.string_alias = object()
# uncomment to see failure
# resource.save(force_admin=True)

This is a drop-in replacement for `Resource` and `Tile` `save()` methods, so it performs side effects like indexing and writing to the edit log.

In [16]:
from arches.app.models.models import EditLog
EditLog.objects.filter(resourceinstanceid=resource.pk).count()

6

#### Deleting
Just call `delete()`.

> [!WARNING]
> [Known issue](https://github.com/archesproject/arches-querysets/issues/35) that tiles should remove themselves from their aliased_data containers.

In [17]:
resource.aliased_data.datatypes_1.delete()  # shows cascade deletes on related objects

(7,
 {'models.ResourceXResource': 2,
  'arches_querysets.TileTree': 1,
  'models.File': 3,
  'models.TileModel': 1})

#### Creating
Call `append_tile()` with a nodegroup alias to append a single blank tile populated with default values, or call the low-level `TileTree.create_blank_tile()` classmethod to do more surgical inserts:

In [18]:
resource.refresh_from_db()
resource.append_tile("datatypes_1")
resource.aliased_data.datatypes_1.aliased_data.non_localized_string_alias

'The answer to life, the universe, and everything.'

Flesh out an existing resource with one additional placeholder tile at every empty nodegroup, with default node values populated.

>[!INFO]
> [Known issue:](https://github.com/archesproject/arches-querysets/issues/109) currently only works for existing resources.

In [19]:
resource.refresh_from_db()
resource.fill_blanks()
resource.aliased_data

namespace(datatypes_1=<TileTree: datatypes_1 (2a168165-101f-4f8d-a4a6-6230472fa6d1)>,
          datatypes_n=[<TileTree: datatypes_n (7b7d4c9f-e809-47d6-91be-bcce161f2f5a)>])

#### Tailoring the results

Just like ordinary Django usage, you can avoid the overhead of instantiating model instances (and fetching graph objects to aid with display value calculations) by requesting just `.values()`:

In [20]:
results = TileTree.objects.get_tiles(
    "datatype_lookups", nodegroup_alias="datatypes_n"
).values("number_alias_n", "concept_list_alias_n")
for result in results:
    print(result)

{'number_alias_n': 42.0, 'concept_list_alias_n': ['d8c60bf4-e786-11e6-905a-b756ec83dad5']}
{'number_alias_n': None, 'concept_list_alias_n': None}


Aggregate functions from Django are available:

In [21]:
from django.db.models import Max
TileTree.objects.get_tiles("datatype_lookups", "datatypes_1").aggregate(my_max=Max("number_alias"))

{'my_max': None}

You can limit the depth of the child tiles that are fetched. The default is 20.

In [22]:
resource = ResourceTileTree.objects.get_tiles("datatype_lookups", depth=0).filter(number_alias_n=[42]).get()
print(resource.aliased_data.datatypes_n[0].aliased_data.datatypes_n_child)

# Try again with default depth (20)
resource = ResourceTileTree.objects.get_tiles("datatype_lookups").filter(number_alias_n=[42]).get()
print(resource.aliased_data.datatypes_n[0].aliased_data.datatypes_n_child)

[]
[<TileTree: datatypes_n_child (fcd8f7bd-10c1-4b2b-97e7-1cc0caed7783)>]


You can provide node values as a subquery targeting anything else in the database:

In [23]:
from arches.app.models.models import Value

from django.contrib.postgres.aggregates import ArrayAgg
from django.db.models import Q

concepts_in_use = TileTree.objects.get_tiles(
    "datatype_lookups", "datatypes_n"
).aggregate(
    concepts_in_use=(ArrayAgg("concept_alias_n", filter=Q(concept_alias_n__isnull=False)))
)["concepts_in_use"]

Value.objects.filter(pk__in=concepts_in_use)

<QuerySet [<Value: Value object (d8c60bf4-e786-11e6-905a-b756ec83dad5)>]>

#### API Integration

arches-querysets includes an integration with one API client: [`djangorestframework`](https://www.django-rest-framework.org/). Be sure to include the `[drf]` in `pip install arches-querysets[drf]` to pull in the optional dependency. Integrations with other Django API frameworks should be possible.

Using a framework allows you to reach for more sophistication as your needs evolve. DRF encouages a declarative programming style with classes, but these can be altered at runtime by overriding specific hooks. Here are the basic layers and how they invoke each other:

> [!NOTE]
> The [classy DRF docs](https://cdrf.co) are helpful to see the methods and corresponding sources for generic views.

#### Routes
Manual. There has been no investigation into extending DRF's router functionality to auto generate routes from models.

See default [routes](https://github.com/archesproject/arches-querysets/blob/main/arches_querysets/urls.py)

You can use Django's router to invoke different views based on the presence or absence of route components.

#### Views

arches-querysets ships with four generic views to handle basic CRUD operations on resources and tiles: resource collection (list/create), resource detail, tile collection (list/create), and tile detail.
```py
class ArchesResourceListCreateView(ArchesModelAPIMixin, ListCreateAPIView):
    permission_classes = [ResourceEditor | ReadOnly]
    serializer_class = ArchesResourceSerializer
    parser_classes = [JSONParser, MultiPartJSONParser]
    pagination_class = ArchesLimitOffsetPagination
```

You can compose your own views declaratively by subclassing these ones. The basic API functionality is provided by the `*APIView` from DRF. Again, I recommend [classy DRF docs](https://cdrf.co) to see what is available.

Arches-specific logic is encapsulated in `ArchesModelAPIMixin` to be able to be composed into any generic view you create yourself.

The serializer declared on the class, but can be altered via `get_serializer_class` or `get_serializer` as the "blank" utility routes do:

In this example, a different serializer is returned according to the value of a query param.

```py
class ArchesResourceBlankView(ArchesModelAPIMixin, RetrieveAPIView):
    permission_classes = [ReadOnly]
    serializer_class = ArchesResourceSerializer

    def get_serializer_class(self):
        if self.request.GET.get("exclude_children", "").lower() == "true":
            return ArchesResourceTopNodegroupsSerializer
        return self.serializer_class

    def get_object(self, *args, **kwargs):
        return None
```


#### Serializers (API forms)

Serializers are forms. They are the sanitization layer that validates that submitted data is in the correct shape and coerces types to a single format so that downstream code doesn't have to guess whether a variable is a `string` or a `UUID`, etc.

Serializers are an API schema. They are where you declare what fields exist, which ones are writable, read-only, have defaults, etc., and what representation you get (e.g. extra properties, discussed later).

DRF allows you to write a serializer from scratch:

```py
class MySerializer(serializer.Serializer):
    first_name = fields.CharField()
```

But if `first_name` is a field on your Django model, that's repetitious and risks drift. Use a `ModelSerializer` to *infer* the serializer fields from your Django model fields. You can then add or override fields as you please, either in full, at the class-level, or in part, by passing surgical updates via `extra_kwargs`.

Here is an example that does all of the above:

```py
class ArchesResourceSerializer(serializers.ModelSerializer, NodeFetcherMixin):
    # aliased_data is a virtual field not inferred by serializers.ModelSerializer.
    aliased_data = ResourceAliasedDataSerializer(required=False, allow_null=False)

    # Custom read-only fields.
    graph_has_different_publication = serializers.SerializerMethodField()

    class Meta:
        model = ResourceTileTree
        ...
        # TODO (arches_version): when Arches 8.1 is the lowest supported version,
        # read_only_fields can be removed, since at that point we can depend on
        # the equivalent editable=False on the model fields (done in 8.1).
        read_only_fields = (
            "principaluser",
            "name",
            "descriptors",
            "legacyid",
            "graph_publication",
            "resource_instance_lifecycle_state",
        )
        extra_kwargs = {
            "resourceinstanceid": {"initial": uuid.uuid4, "allow_null": True},
            "graph": {"allow_null": True},
        }
```

In a classic DRF setup, you would do all your validation in the serializer layer. arches-querysets is an adaptation tool for arches deployments that have entry points besides this API framework, so we delegate a lot of the actual node value validation to the existing arches datatype methods. But DRF still provides some helpful early failures if the wrong python types are submitted, e.g. a `list` instead of a `dict`.

If you choose to use DRF-level validation, implement methods with `validate_{field_name}`:

```py
class LingoTileSerializer(ArchesTileSerializer):
    def validate_appellative_status(self, data, initial_tile_data):
        if data:
            new_label_lang = None
            new_label_type = None
            if new_label_languages := data.appellative_status_ascribed_name_language:
                new_label_lang = new_label_languages[0]
            if new_label_types := data.appellative_status_ascribed_relation:
                new_label_type = new_label_types[0]

            if new_label_lang and new_label_type:
                resource_pk = self.instance.resourceinstance_id
                scheme = ResourceTileTree.get_tiles(self.graph_slug).get(pk=resource_pk)
                self._check_pref_label_uniqueness(
                    initial_tile_data, scheme, new_label_lang, new_label_type
                )

        return data
```

Arches data is dynamic, so methods like `get_fields()` and friends are used to generate classes on the fly at runtime.
If you find this behavior to "magical", there is nothing stopping you from hard-coding your serializers. This might work well for small, stable projects.

How do you know what the expected types are? Import and print the serializer:

```py
In [1]: from arches_querysets.rest_framework.serializers import *

In [2]: ArchesTileSerializer(graph_slug="datatype_lookups", nodegroup_alias="datatypes_1")
Out[2]: 
ArchesTileSerializer(graph_slug='datatype_lookups', nodegroup_alias='datatypes_1'):
    tileid = UUIDField(allow_null=True, required=False, validators=[])
    resourceinstance = PrimaryKeyRelatedField(allow_null=True, html_cutoff=0, queryset=<ResourceTileTreeQuerySet [<ResourceTileTree: Arches System Settings:  (a106c400-260c-11e7-a604-14109fd34195)>, <ResourceTileTree: Datatype 
    ...
                    geojson_feature_collection_alias_child = CharField(allow_blank=True, allow_null=True, help_text=None, initial=None, label='', required=False, style={'alias': 'geojson_feature_collection_alias_child', 'visible': True, 'widget_config': <arches.app.models.fields.i18n.I18n_JSON object>, 'datatype': 'geojson-feature-collection', 'sortorder': 0})
                concept_list_alias_child = JSONField(allow_null=True, decoder=None, encoder=None, help_text=None, initial=None, label='', required=False, style={'alias': 'concept_list_alias_child', 'visible': True, 'widget_config': <arches.app.models.fields.i18n.I18n_JSON object>, 'datatype': 'concept-list', 'sortorder': 0})
                number_alias_child = FloatField(allow_null=True, help_text=None, initial=7, label='', required=False, style={'alias': 'number_alias_child', 'visible': True, 'widget_config': <arches.app.models.fields.i18n.I18n_JSON object>, 'datatype': 'number', 'sortorder': 0})
```

Or use the /blank API to see a representation stocked with default values.

How can you monitor what's going on at the serialization boundary? Watch `to_internal_value()` coming in, or `to_representation()` going out.

Here's an example of patching `to_internal_value()` to accept nested writes, which DRF doesn't support out of the box given there is no reasonable default behavior:

```py
    def to_internal_value(self, data):
        """Make nested aliased data writable."""
        self.initial_data = data
        return _handle_nested_aliased_data(data, fields_map=self.fields)
```

Where are the GET, POST handlers, etc?

Answer: given that retrieving an object is needed not only by GETs but also by update operations, the handlers are abstracted away, encouraging you to focus instead on the actions of creating, retrieving, updating, and deleting. See `create()`, `retrieve()`, `update()`, `delete()` for the entire action, and `perform_create()` for 

*You do not need to use serializers inside a request/response cycle.* They're just forms! See `test_bind_data_to_serializer` in the test suite for ingesting static data.

#### QuerySets

The serializer is responsible for fetching a model instance or instances via a queryset.

Serializer fields have declarative arguments for querysets. This one cancels some prefetches that were provided by this model's `get_queryset()` manager method:

```py
parenttile = serializers.PrimaryKeyRelatedField(
    queryset=TileTree.objects.prefetch_related(None),
    required=False,
    allow_null=True,
    html_cutoff=0,
)
```
(`html_cutoff` is useful to make sure the forms that ship with the browsable API are not trying to fetch tons of objects.)

You can create the queryset dynamically in the view by overriding `get_queryset()`, as `ArchesModelAPIMixin` does. This is a bit overloaded, dealing with both Resources and Tiles, but you can imagine factoring this more cleanly if you were willing to ship more combinations of mixins:

```py
def get_queryset(self):
    options = self.serializer_class.Meta
    if issubclass(options.model, ResourceInstance):
        qs = options.model.get_tiles(
            self.graph_slug,
            resource_ids=self.resource_ids,
            as_representation=True,
        ).select_related("graph")
        if arches_version >= (8, 0):
            qs = qs.select_related("resource_instance_lifecycle_state")
    elif issubclass(options.model, TileModel):
        qs = options.model.get_tiles(
            self.graph_slug,
            self.nodegroup_alias,
            as_representation=True,
            resource_ids=self.resource_ids,
        ).select_related("nodegroup", "resourceinstance__graph")
        if self.resource_ids:
            qs = qs.filter(resourceinstance__in=self.resource_ids)
```

The `.get_tiles()` queryset method is described above. It just encapsulates providing node aliases as field aliases to the ORM and scheduling prefetching of related objects necessary for calculating display values or building nested tiles.

Tangent: Remember to cancel all of those additional queries via `.values()` if you are only extracting values and do not need model instances.


#### Searching

Add query params like `aliased_data__string_alias__icontains=foo` to see advanced search functionality. See test_filter_kwargs() and the tested lookups in test_lookups.py.

#### QuerySets under the hood

A final word about how all this works.



#### Tooling

django-debug-toolbar (I use version 5.x) takes minimal installation. Add URLs and middleware and installed app, and a toolbar appears that will show SQL queries in a waterfall visual format, with buttons to expand to see explain plans, results, and where in the source the query was triggered. It also flags N+1 (repetitive) queries.

It's designed to work for HTML responses, but you change the middleware to get it to work with JSON responses, if you want to avoid profiling things out of scope of your API. See this [gist](https://gist.github.com/fabiosussetto/c534d84cbbf7ab60b025?permalink_comment_id=4581191#gistcomment-4581191)

For the adventurous, you can try throwing exceptions when your project issues an N+1 query by:
- checking out this [PR](https://github.com/django/django/pull/17554) to Django
- changing the default `self._fetch mode` in django/db/models/query.py from `FETCH_ONE` to `RAISE` (or use it as designed, and use model managers, but that's a larger refactor)
- YMMV given that you will likely need to hack around failures in core arches

This [profile decorator](https://adamj.eu/tech/2023/07/23/python-profile-section-cprofile/) is also amazing. I use the context manager / function decorator version.